In [ ]:
import random

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt


def plot_hands3d(ax, points, color, linewidth='3'):
    # Add bone connections
    bones = [(13, 11),
             (13, 12),
             (13, 1),
             (0, 1),
             (13, 3),
             (2, 3),
             (13, 5),
             (4, 5),
             (13, 7),
             (6, 7),
             (13, 10),
             (9, 10),
             (8, 9)]

    for connection in bones:
        coord1 = points[connection[0]]
        coord2 = points[connection[1]]
        coords = np.stack([coord1, coord2])
        ax.plot(coords[:, 0], coords[:, 1], coords[:, 2], c=color, linewidth=linewidth)

# NYU Hand Pose Dataset

The [NYU Hand Pose Dataset](https://jonathantompson.github.io/NYU_Hand_Pose_Dataset.htm) was published in 2014. It provides 72,757 training frames and 8,252 test frames captured with RGB-D cameras (Kinect 1). Keypoint annotations are provided for each frame.

In [ ]:
# Load dataset
testing_mat = sio.loadmat("/Users/sayer/Downloads/test_joint_data.mat")
training_mat = sio.loadmat("/Users/sayer/Downloads/train_joint_data.mat")
test_dataset = testing_mat['joint_uvd'][0]
train_dataset = training_mat['joint_uvd'][0]

# Show a sample
eval_joints = [0, 3, 6, 9, 12, 15, 18, 21, 24, 25, 27, 30, 31, 32]
train_dataset = train_dataset[:, eval_joints]
test_dataset = test_dataset[:, eval_joints]

In [ ]:
sample = train_dataset[10000]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(-90, -90)
plot_hands3d(ax, sample, 'b')

# Data Normalization

Normalization is extremely important when comparing two samples of any data type. Consider a model that detects the keypoints of a hand from two separate images. Depending on the relative location of the hand in the image, the coordinates will vary greatly.

In [ ]:
x1 = train_dataset[0]
x2 = train_dataset[10000]

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(-90, -90)
plot_hands3d(ax, x1, 'b')
plot_hands3d(ax, x2, 'r')

Normalization can resolve the change in relative size so that a fair comparison can be made. For this particular dataset, we will normalize each sample based on its size and set the origin to be the center of the palm.

In [ ]:
def normalize_hand(points):
    """Normalize the hand and center it on the palm"""

    center = points[-1]  # The palm keypoint
    scale_factor = 1.0 / (points.max(0)[0] - points.min(0)[0])
    norm_points = points.copy()
    norm_points -= center
    norm_points *= scale_factor

    return norm_points

In [ ]:
x1_norm = normalize_hand(x1)
x2_norm = normalize_hand(x2)

fig1 = plt.figure()
ax1 = fig1.add_subplot(111, projection='3d')
ax1.view_init(-90, -90)
plot_hands3d(ax1, x1_norm, 'b')
plot_hands3d(ax1, x2_norm, 'r')

# Finding the Closest Match

In this notebook, your task is to complete the function `find_closest_match` to retrieve samples in the dataset that most closely match an input sample based on either L1 or L2 loss.

**Requirements:**
1. Define two functions to compute L1 and L2 loss.
2. Normalize the input sample and each sample in the dataset.
3. Compute the loss between the input sample and each sample in the dataset.
4. Return the index of the sample in the dataset that has the lowest loss.

In [ ]:
def find_closest_match(x, dataset, loss_fn):
    """Searches the `dataset` for the sample that has the lowest
    error as compared with `loss_fn`.
    """
  x_norm = normalize_hand(x)
    
    min_loss = float('inf')
    closest_sample = None
    
    for sample in dataset:
        sample_norm = normalize_hand(sample)
        loss = loss_fn(x_norm, sample_norm)
        if loss < min_loss:
            min_loss = loss
            closest_sample = sample
    
    return closest_sample

# Testing Your Implementation

Be sure to fill in the definitions for `l1_loss`, `l2_loss`, and `find_closest_match`. You can then run the rest of the notebook to visualize the results of your implementation.

In [ ]:
def l1_loss(x, y):
    """Compute the L1 loss between two samples."""
    return 0

def l2_loss(x, y):
    """Compute the L2 loss between two samples."""
    return 0

# Search for the closest match
input_idx = random.randint(0, len(test_dataset) - 1)
input_sample = test_dataset[input_idx]
l1_match = find_closest_match(input_sample, train_dataset, l1_loss)

# Compute using L2 loss
l2_match = find_closest_match(input_sample, train_dataset, l2_loss)

# Normalize samples
input_sample = normalize_hand(input_sample)
l1_match = normalize_hand(l1_match)
l2_match = normalize_hand(l2_match)

# Visualize Results
fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d')
ax1.set_title('L1 Match (Distance: {:.2f})'.format(l1_loss(input_sample, l1_match)))
ax1.legend(['Input', 'Match'])
ax2 = fig.add_subplot(122, projection='3d')
ax2.set_title('L2 Match (Distance: {:.2f})'.format(l2_loss(input_sample, l2_match)))
plot_hands3d(ax1, input_sample, 'b')
plot_hands3d(ax1, l1_match, 'r')
plot_hands3d(ax2, input_sample, 'b')
plot_hands3d(ax2, l2_match, 'r')